In [1]:
# import nltk libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
# Download nltk packages
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\R
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\R
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# import numpy and keras packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [4]:
# load dataset and some empty array
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('queans.json').read()
intents = json.loads(data_file)

In [5]:
intents

{'queans': [{'tag': 'greeting',
   'questions': ['Hi', 'Hi there', 'Is anyone there?', 'Hey', 'Hola', 'Hello'],
   'answers': ['Hey, how can I help?',
    'Hello',
    'Good to see you again',
    'Hi there, what can I do?'],
   'context': ['']},
  {'tag': 'care',
   'questions': ['How are you?',
    'How are you doing?',
    "What's up?",
    'How are you going?'],
   'answers': ['I am fine, what about you?',
    'I am great thanks.',
    "It's going not to badly at all, thank you.",
    'I am good.'],
   'context': ['']},
  {'tag': 'name',
   'questions': ['What is your name?',
    'Who are you?',
    'Tell me your name?',
    'Can you tell me your name?',
    'What should i call you?'],
   'answers': ['You can call me Bot.', 'I am Bot.', 'My name is Bot.'],
   'context': ['']},
  {'tag': 'age',
   'questions': ['How old are you?', 'How old is Bot?', 'What is your age?'],
   'answers': ['I am 20 years old!', '20 years young!'],
   'context': ['']},
  {'tag': 'goodbye',
   'questions'

In [6]:
for intent in intents['queans']:
    for pattern in intent['questions']:
        
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # add documents in corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
        

In [7]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

#documents = combination b/w patterns and intents
print(len(documents), "documents")

# classes = intents
print(len(classes), "classes", classes)

# word = all words, vocabulary
print(len(words), "unique lemmatized words", words)

74 documents
17 classes ['adverse_drug', 'age', 'blood_pressure', 'blood_pressure_search', 'boss', 'care', 'data', 'fun', 'goodbye', 'greeting', 'hospital_search', 'name', 'options', 'pharmacy_search', 'question', 'sukariya', 'test']
113 unique lemmatized words ["'s", 'a', 'adverse', 'age', 'all', 'am', 'anyone', 'are', 'ask', 'be', 'behavior', 'blood', 'bos', 'bot', 'by', 'bye', 'call', 'can', 'causing', 'check', 'could', 'created', 'cye', 'data', 'day', 'detail', 'do', 'doing', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'going', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'human', 'i', 'id', 'is', 'joke', 'know', 'later', 'leaving', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'made', 'management', 'me', 'module', 'more', 'my', 'name', 'nearby', 'of', 'offered', 'old', 'one', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'query', 'question', 'questons', 'reaction', 'related', 'result', 'robot', 'save'

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [9]:
# creating our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

In [10]:
# training set, bag of words for each sentence
for doc in documents:
     # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [11]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

In [12]:
# create train and test lists. x - patterns, y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [13]:
# Create model - 3 layers. First layer 128 neurons and second layer 64 neurons and
# 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

In [14]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [15]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
74/74 [==============================] - 0s 6ms/step - loss: 2.8538 - accuracy: 0.0676
Epoch 2/200
74/74 [==============================] - 0s 443us/step - loss: 2.7996 - accuracy: 0.1216
Epoch 3/200
74/74 [==============================] - 0s 211us/step - loss: 2.7367 - accuracy: 0.1216
Epoch 4/200
74/74 [==============================] - 0s 211us/step - loss: 2.5852 - accuracy: 0.2297
Epoch 5/200
74/74 [==============================] - 0s 211us/step - loss: 2.4440 - accuracy: 0.3378
Epoch 6/200
74/74 [==============================] - 0s 422us/step - loss: 2.4420 - accuracy: 0.2973
Epoch 7/200
74/74 [==============================] - 0s 211us/step - loss: 2.2874 - accuracy: 0.3649
Epoch 8/200
74/74 [==============================] - 0s 211us/step - loss: 2.1910 - accuracy: 0.3514
Epoch 9/200
74/74 [==============================] - 0s 508us/step - loss: 1.9456 - accuracy: 0.4730
Epoch 10/200
74/74 [==============================] - 0s 314us/step - loss: 1.8400 - accuracy

Epoch 81/200
74/74 [==============================] - 0s 364us/step - loss: 0.1707 - accuracy: 0.9459
Epoch 82/200
74/74 [==============================] - 0s 404us/step - loss: 0.0659 - accuracy: 0.9730
Epoch 83/200
74/74 [==============================] - 0s 364us/step - loss: 0.0821 - accuracy: 0.9865
Epoch 84/200
74/74 [==============================] - 0s 310us/step - loss: 0.1161 - accuracy: 0.9595
Epoch 85/200
74/74 [==============================] - 0s 323us/step - loss: 0.1324 - accuracy: 0.9865
Epoch 86/200
74/74 [==============================] - 0s 323us/step - loss: 0.0690 - accuracy: 0.9865
Epoch 87/200
74/74 [==============================] - 0s 310us/step - loss: 0.1016 - accuracy: 0.9730
Epoch 88/200
74/74 [==============================] - 0s 323us/step - loss: 0.1594 - accuracy: 0.9595
Epoch 89/200
74/74 [==============================] - 0s 323us/step - loss: 0.0697 - accuracy: 0.9730
Epoch 90/200
74/74 [==============================] - 0s 323us/step - loss: 0.1503

74/74 [==============================] - 0s 377us/step - loss: 0.0599 - accuracy: 0.9865
Epoch 161/200
74/74 [==============================] - 0s 350us/step - loss: 0.0209 - accuracy: 1.0000
Epoch 162/200
74/74 [==============================] - 0s 350us/step - loss: 0.0821 - accuracy: 0.9595
Epoch 163/200
74/74 [==============================] - 0s 391us/step - loss: 0.0379 - accuracy: 1.0000
Epoch 164/200
74/74 [==============================] - 0s 472us/step - loss: 0.0419 - accuracy: 0.9865
Epoch 165/200
74/74 [==============================] - 0s 337us/step - loss: 0.0094 - accuracy: 1.0000
Epoch 166/200
74/74 [==============================] - 0s 310us/step - loss: 0.0741 - accuracy: 0.9730
Epoch 167/200
74/74 [==============================] - 0s 323us/step - loss: 0.0700 - accuracy: 0.9595
Epoch 168/200
74/74 [==============================] - 0s 310us/step - loss: 0.0136 - accuracy: 1.0000
Epoch 169/200
74/74 [==============================] - 0s 323us/step - loss: 0.0630 - a